In [7]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

# train.csv 파일 불러오기
train_data = pd.read_csv('train.csv')

# 결측치를 평균값으로 대체
train_data = train_data.fillna(train_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
encoder = OneHotEncoder(sparse=False)
time_encoded = encoder.fit_transform(train_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
train_data = pd.concat([train_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 풍속을 예측할 특성(입력 변수)과 풍속(출력 변수)을 분리합니다.
X_train = train_data.drop(['ID', '풍속 (m/s)'], axis=1)  # 입력 변수들
y_train = train_data['풍속 (m/s)']  # 출력 변수 (풍속)

# 엑스트라 트리 모델을 사용하여 풍속 예측 모델을 학습시킵니다.
model = ExtraTreesRegressor(random_state=42)  # 모델 변경: RandomForestRegressor -> ExtraTreesRegressor
model.fit(X_train, y_train)

# test.csv 파일 불러오기
test_data = pd.read_csv('test.csv')

# 결측치를 평균값으로 대체
test_data = test_data.fillna(test_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
time_encoded = encoder.transform(test_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
test_data = pd.concat([test_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 테스트 데이터로 풍속 예측을 수행합니다.
X_test = test_data.drop('ID', axis=1)  # 테스트 입력 변수들

# 여러 개의 엑스트라 트리 모델로 예측값을 구합니다.
num_models = 5  # 여러 개의 모델 사용
y_preds = np.zeros((X_test.shape[0], num_models))

for i in range(num_models):
    model = ExtraTreesRegressor(random_state=42)
    y_preds[:, i] = model.fit(X_train, y_train).predict(X_test)

# 예측값들을 평균하여 최종 예측값을 구합니다.
y_pred = np.mean(y_preds, axis=1)

# Submit / 제출
submission = pd.read_csv('./sample_submission.csv')
submission['풍속 (m/s)'] = y_pred

# 예측 결과를 submission.csv 양식에 맞게 저장합니다.
submission.to_csv('submission.csv', index=False)

print("풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.")

# 성능 평가 (RMSE, MAE)를 수행합니다.
train_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, train_pred))
mae = mean_absolute_error(y_train, train_pred)
print("평균 RMSE:", rmse)
print("평균 MAE:", mae)


C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecat

풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.
평균 RMSE: 3.604293716263166e-15
평균 MAE: 2.4791037949132187e-15
